#

In [59]:
using AlgebraOfGraphics
using AlgebraOfGraphics: density
using CairoMakie
using DataFrames
using DataFramesMeta
using Arrow
using PartialFunctions
using LaTeXStrings
using RCall
using Statistics

set_aog_theme!()

In [242]:
round(Int, 1.00)

1

In [243]:
function load(path::String)
    df = path |> Arrow.Table |> DataFrame |> dropmissing
    
    # if 'radial_distance' is in the dataframe, round it
    if "radial_distance" in names(df)
        df = @chain df begin
            @transform :r = string.(round.(Int, :radial_distance))
        end
    end

    df = @chain df begin
        @transform(
            :j0_k = abs.(:j0_k),
            :j0_k_norm = abs.(:j0_k_norm),
        )
    end

    # Iterate through each column in the DataFrame
    filter(row -> all(x -> !(x isa Number && isnan(x)), row), df)
end

load (generic function with 1 method)

In [244]:
begin
    dir = "../data/05_reporting"

    j_events_low_fit = load("$dir/events.JNO.fit.ts_1.00s_tau_60s.arrow")
    j_events_high_fit = load("$dir/events.JNO.fit.ts_0.12s_tau_60s.arrow")
    j_events_low_der = load("$dir/events.JNO.derivative.ts_1.00s_tau_60s.arrow")
    j_events_high_der = load("$dir/events.JNO.derivative.ts_0.12s_tau_60s.arrow")
    w_events = load("$dir/events.Wind.fit.ts_0.09s_tau_60s.arrow")

    # add a label column to the dataframes
    j_events_low_fit[!, :label] .= "1 Hz (fit)"
    j_events_high_fit[!, :label] .= "8 Hz (fit)"
    j_events_low_der[!, :label] .= "1 Hz (derivative)"
    j_events_high_der[!, :label] .= "8 Hz (derivative)"

    # filter high time resolution events
    j_events_high_fit = @chain j_events_high_fit begin
        filter(:len => >(240), _)
    end

    j_events_der = vcat(j_events_low_der, j_events_high_der)

    # combine the dataframes
    j_events = vcat(j_events_low_fit, j_events_high_fit)

    j_events = @chain j_events begin
        filter(:"fit.stat.rsquared" => >(0.95), _)
    end

    names(j_events)
end

94-element Vector{String}:
 "time"
 "tstart"
 "tstop"
 "t.d_end"
 "t.d_start"
 "t.d_time"
 "index_diff"
 "len"
 "std"
 "std_prev"
 ⋮
 "v.Alfven.before.l"
 "v.Alfven.after.l"
 "n.change"
 "v.ion.change.l"
 "B.change"
 "v.Alfven.change"
 "v.Alfven.change.l"
 "r"
 "label"

In [106]:
log_axis = (yscale=log10, xscale=log10)

begin

    l_lab = L"Thickness ($km$)"
    l_norm_lab = L"Normalized Thickness ($d_i$)"
    j_norm_lab = L"Normalized Current Density ($J_A$)"

    di_map = :ion_inertial_length => L"Ion Inertial Length ($km$)"
    di_log_map = :ion_inertial_length => log10 => L"Log Ion Inertial Length ($km$)"
    l_map = :L_k => l_lab
    l_log_map = :L_k => log10 => L"Log %$l_lab"
    l_norm_map = :L_k_norm => l_norm_lab
    l_norm_log_map = :L_k_norm => log10 => L"Log %$l_norm_lab"
    
    jA_map = :j_Alfven => L"Alfvénic Current Density ($nA/m^2$)"
    jA_log_map = :j_Alfven => log10 => L"Log Alfvénic Current Density ($nA/m^2$)"
    j_map = :j0_k => L"Current Density ($nA/m^2$)"
    j_log_map = :j0_k => log10 => L"Log Current Density ($nA/m^2$)"
    j_norm_map = :j0_k_norm => L"Normalized Current Density ($J_A$)"

    j_norm_log_map = :j0_k_norm => log10 => L"Log %$j_norm_lab"

    j_limit = (10^-1, 10^1)
    j_norm_limit = (10^-2, 2)
end


(0.010000000000000002, 2)

In [64]:
begin
    fig = Figure(size=(1000, 800))

    datalayer = mapping() * visual(Scatter, markersize=1, color=:white, alpha=0.1)

    begin
        layer = histogram(bins=range(1, 5, length=64), normalization=:pdf)
        plt = layer * mapping(di_log_map, l_log_map)
        plt1 = data(j_events) * mapping(layout=:r) * plt
        plt2 = data(w_events) * plt
    
        l_log_limit = ((1.5, 3.5), (1.5, 4.5))
        axis = (;limits=l_log_limit)
        draw!(fig[1:2,1:3], plt1, axis=axis)
        draw!(fig[1:2,4:5], plt2, axis=axis) 
    end

    # Current Density Panels
    begin
        layer = histogram(bins=range(-2, 3, length=64), normalization=:pdf)
        plt = layer * mapping(jA_log_map, j_log_map)
        plt3 = data(j_events) * mapping(layout=:r) * plt
        plt4 = data(w_events) * plt
    
        j_log_limit = ((-1, 3), (-2, 2))
        axis = (;limits=j_log_limit)
        draw!(fig[3:4,1:3], plt3, axis=axis)
        draw!(fig[3:4,4:5], plt4, axis=axis)
    end

    # Make ablines across different r panels
    # begin
        # ablines_data = (; intercepts=[-3,-1,1], slopes=[1,1,1]) 
        # lines = data(ablines_data) * mapping(:intercepts, :slopes) * visual(ABLines, linestyle=:dash)
        # draw!(fig[3:4,1:3], lines)
    # end

    Label(fig[0,1:3], "Juno in Different Radial Distances", fontsize=20)
    Label(fig[0,4:5], "Wind 11 Hz", fontsize=20)

    fig
end

In [65]:
begin
    fig = Figure(size=(1000, 800))

    pdflayer = density() * visual(Contour)
    # small scatter points
    datalayer = mapping() * visual(Scatter, markersize=3)

    # layer = pdflayer + datalayer
    layer = datalayer
    plt = layer * mapping(di_map, l_map)
    plt1 = data(j_events) * mapping(layout=:r) * plt
    plt2 = data(w_events) * plt

    l_limit = ((10^1, 10^5), (10^1, 10^5))
    axis = merge(log_axis, (;limits=l_limit))
    draw!(fig[1:2,1:3], plt1, axis=axis)
    draw!(fig[1:2,4:5], plt2, axis=axis)

    plt = layer * mapping(jA_map, j_map)
    plt3 = data(j_events) * mapping(layout=:r) * plt
    plt4 = data(w_events) * plt

    j_limit = ((10^-1, 10^3), (10^-2, 10^2))
    axis = merge(log_axis, (;limits=j_limit))
    draw!(fig[3:4,1:3], plt3, axis=axis)
    draw!(fig[3:4,4:5], plt4, axis=axis)


    Label(fig[0,1:3], "Juno in Different Radial Distances", fontsize=20)
    Label(fig[0,4:5], "Wind 11 Hz", fontsize=20)

    fig
end

In [66]:
begin
    fig = Figure(size=(1000, 800))
    datalimits_f = x -> quantile(x, [0.05, 0.95])

    begin
        layer = histogram(bins=24, datalimits=datalimits_f, normalization=:pdf)
        plt = layer * mapping(l_norm_log_map, j_norm_log_map)
        plt1 = data(j_events) * mapping(layout=:r) * plt
        plt2 = data(w_events) * plt

        axis = (;)
        draw!(fig[1:2, 1:3], plt1, axis=axis)
        draw!(fig[1:2, 4:5], plt2, axis=axis)
    end

    Label(fig[0, 1:3], "Juno in Different Radial Distances", fontsize=20)
    Label(fig[0, 4:5], "Wind 11 Hz", fontsize=20)

    fig
end

fig

In [109]:
function plot_l_j_r(df)
    fig = Figure(size=(1000, 1000))

    plt = data(df) * mapping(col=:r, color=:label)

    plt1 = plt * mapping(l_map) * density(datalimits=((0, 5000),))
    plt2 = plt * mapping(l_norm_map) * density(datalimits=((0, 50),))
    plt3 = plt * mapping(j_map) * density(datalimits=(j_limit,))
    plt4 = plt * mapping(j_norm_map) * density(datalimits=(j_norm_limit,))

    axis = (; yscale = identity)
    # NOTE: log axis for density is not working now
    # axis = (; yscale=log10, limits=(nothing, nothing, 10^-3, 1))
    # l_axis = (; yscale = log10, limits=(nothing, (10^-5, 10^-3)))
    # l_norm_axis = (; yscale = log10, limits=(nothing, (10^-3, 1)))

    grid1 = draw!(fig[1, 1:5], plt1, axis=axis)
    grid2 = draw!(fig[2, 1:5], plt2, axis=axis)
    grid3 = draw!(fig[3, 1:5], plt3, axis=axis)
    grid4 = draw!(fig[4, 1:5], plt4, axis=axis)
    legend!(fig[0, 1:end], grid1, titleposition=:left, orientation=:horizontal)

    fig
end

fig = plot_l_j_r(j_events_der)
fig

In [254]:
@rput j_events_der
@rput j_events

R"""
library(ggplot2)
library(ggpubr)
library(patchwork)
source('utils.R')
"""

R"""

density_plot <- function(df, x, facet.by = "r", strip.position = "top"){
  p <- ggdensity(
    df, 
    x = x, color = "label",
    facet.by = facet.by,
    add = "median", rug = TRUE
  ) %>%
  facet(facet.by, ncol = 1, strip.position = strip.position)
}

plot <- function(
  df,
  j0_k_ulim = 10
) {

  p1 <- density_plot(
    filter(df, L_k < 8000), "L_k"
  )
  
  p2 <- density_plot(
    filter(df, L_k_norm < 50), "L_k_norm"
  )
  
  p3 <- density_plot(
    filter(df, j0_k < j0_k_ulim), "j0_k"
  )
  
  p4 <- density_plot(
    filter(df, j0_k_norm < 2), "j0_k_norm", strip.position = "right"
  )
  
  p1 <- ggpar(p1, 
      yscale="log10", ylim = c(10^-5, 10^-3)
    ) + labs(x = lab_l)
  
  p2 <- ggpar(p2, 
      yscale="log10", ylim = c(10^-3, 1), ylab=FALSE
    ) + labs(x = lab_l_norm)
  
  p3 <- ggpar(p3,
      yscale="log10", ylim = c(10^-2, 10), ylab=FALSE
    ) + labs(x = lab_j)
  
  p4 <- ggpar(p4,
      yscale="log10", ylim = c(10^-2, 10), ylab=FALSE
    ) + labs(x = lab_j_norm)
  
  tag_pool <- unique(df$r)
  
  p1 <- tag_facet(p1, open = "(a.", tag_pool = tag_pool)
  p2 <- tag_facet(p2, open = "(b.", tag_pool = tag_pool)
  p3 <- tag_facet(p3, open = "(c.", tag_pool = tag_pool)
  p4 <- tag_facet(p4, open = "(d.", tag_pool = tag_pool) + theme(
    strip.background = element_blank(),
    strip.text.x = element_blank()
  )

  p <- p1 + p2 + p3 + p4
  p <- p + plot_layout(nrow = 1, guides = "collect") & theme(legend.position='top')  
}
"""


RObject{ClosSxp}
function (df, j0_k_ulim = 10) 
{
    p1 <- density_plot(filter(df, L_k < 8000), "L_k")
    p2 <- density_plot(filter(df, L_k_norm < 50), "L_k_norm")
    p3 <- density_plot(filter(df, j0_k < j0_k_ulim), "j0_k")
    p4 <- density_plot(filter(df, j0_k_norm < 2), "j0_k_norm", 
        strip.position = "right")
    p1 <- ggpar(p1, yscale = "log10", ylim = c(10^-5, 10^-3)) + 
        labs(x = lab_l)
    p2 <- ggpar(p2, yscale = "log10", ylim = c(10^-3, 1), ylab = FALSE) + 
        labs(x = lab_l_norm)
    p3 <- ggpar(p3, yscale = "log10", ylim = c(10^-2, 10), ylab = FALSE) + 
        labs(x = lab_j)
    p4 <- ggpar(p4, yscale = "log10", ylim = c(10^-2, 10), ylab = FALSE) + 
        labs(x = lab_j_norm)
    tag_pool <- unique(df$r)
    p1 <- tag_facet(p1, open = "(a.", tag_pool = tag_pool)
    p2 <- tag_facet(p2, open = "(b.", tag_pool = tag_pool)
    p3 <- tag_facet(p3, open = "(c.", tag_pool = tag_pool)
    p4 <- tag_facet(p4, open = "(d.", tag_pool = tag_pool) + 
        t

In [255]:
R"""
plot(j_events_der)
save_plot("l_j_r_der", width = 15, height = 10)

plot(j_events, j0_k_ulim=10)
save_plot("l_j_r_fit", width = 15, height = 10)
"""

┌ Warning: RCall.jl: Warning: Transformation introduced infinite values in continuous y-axis
│ Warning: Transformation introduced infinite values in continuous y-axis
│ Warning: Transformation introduced infinite values in continuous y-axis
│ Warning: Transformation introduced infinite values in continuous y-axis
│ Warning: Transformation introduced infinite values in continuous y-axis
│ Warning: Transformation introduced infinite values in continuous y-axis
│ Warning: Transformation introduced infinite values in continuous y-axis
│ Warning: Transformation introduced infinite values in continuous y-axis
└ @ RCall /Users/zijin/.julia/packages/RCall/dDAVd/src/io.jl:172
┌ Warning: RCall.jl: Warning: Transformation introduced infinite values in continuous y-axis
│ Warning: Transformation introduced infinite values in continuous y-axis
│ Warning: Transformation introduced infinite values in continuous y-axis
│ Warning: Transformation introduced infinite values in continuous y-axis
│ Warning

RObject{StrSxp}
../figures/l_j_r_fit.pdf

[l_j_r_fit](../figures/l_j_r_fit.png)

[l_j_r_der](../figures/l_j_r_der.png)

In [54]:
function plot_l_r_one(df)
    fig = Figure(size=(1000, 500))

    plt = data(df) * mapping(col=:label, color=:r)
    
    grid1 = draw!( fig[1, 1:2], plt * mapping(:L_k_norm) * density(datalimits=((0, 50),)) )
    grid2 = draw!( fig[2, 1:2], plt * mapping(:L_k) * density(datalimits=((0, 5000),)) )
    legend!(fig[0, 1:end], grid1, titleposition=:left, orientation=:horizontal)

    fig
end

plot_l_r_one(j_events)

In [55]:
function plot_l_r_one(df)
    fig = Figure(size=(1000, 500))

    plt = data(df) * mapping(col=:label, color=:r)
    
    grid1 = draw!( fig[1, 1:2], plt * mapping(:L_k_norm) * density(datalimits=((0, 50),)) )
    grid2 = draw!( fig[2, 1:2], plt * mapping(:L_k) * density(datalimits=((0, 5000),)) )
    legend!(fig[0, 1:end], grid1, titleposition=:left, orientation=:horizontal)

    fig
end

plot_l_r_one(j_events_der)

In [66]:
plt = data(j_events) * mapping(:radial_distance, :Alfven_speed, color=:label) * (visual(Scatter) + smooth())
plt |> draw

In [64]:
plt = data(j_events) * mapping(:radial_distance, jA_map, color=:label) * (visual(Scatter) + smooth())
plt |> draw

In [79]:
using Statistics

In [86]:
# groupby r and describe the data for each group 
# j_events |> @groupby(_.r) |> @map({r=key(_), j0_k=describe(_.j0_k), L_k=describe(_.L_k)})
@chain j_events begin
    groupby(:r)
    combine(:plasma_density =>  mean, :ion_inertial_length => mean, :b_mag => mean) 
end

In [67]:
plt = data(j_events) * mapping(:radial_distance, :plasma_density, color=:label) * (visual(Scatter) + smooth())
plt |> draw

In [65]:
plt = data(j_events) * mapping(:radial_distance, di_map, color=:label) * (visual(Scatter) + smooth())
plt |> draw